# Widening the Space of Similar Values

> A major finding was that the current approaches are considering values that are too similar. This notebook investigates ways to search a wider space.  

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from functools import partial
from pathlib import Path
from typing import Callable, Dict, List, Optional, Iterable, Protocol, Sequence, Tuple, TypeVar, Type

In [ ]:
#| hide
from fastcore.test import *
import math
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import tempfile
import torch
from torch.nn import functional as F
from tqdm.auto import tqdm

In [ ]:
# | hide

from transformer_experiments.common.substring_generator import all_unique_substrings
from transformer_experiments.common.text_analysis import (
    build_next_token_map,
    SubstringFrequencyAnalysis,
    top_nonzero_tokens
)
from transformer_experiments.common.utils import (
    aggregate_by_string_key,
    DataWrapper,
    topk_across_batches,
)
from transformer_experiments.dataset_split import split_text_dataset
from transformer_experiments.datasets.tinyshakespeare import (
    TinyShakespeareDataSet,
)
from transformer_experiments.models.transformer import (
    n_embed,
    n_layer,
    TransformerLanguageModel
)
from transformer_experiments.models.transformer_helpers import (
    unsqueeze_emb,
    EncodingHelpers,
    LogitsWrapper,
    TransformerAccessors
)
from transformer_experiments.trained_models.tinyshakespeare_transformer import (
    create_model_and_tokenizer
)
from transformer_experiments.training_utils import CheckPointer, GetBatchFunction, Trainer
from transformer_experiments.experiments.block_internals import (
    BlockInternalsAccessors,
    BlockInternalsExperiment,
    BatchedBlockInternalsExperiment,
    BlockInternalsAnalysis,
    batch_cosine_sim,
)
from transformer_experiments.experiments.final_ffwd import FinalFFWDExperiment
from transformer_experiments.experiments.similar_strings import (
    SimilarStringsData,
    SimilarStringsExperiment,
    SimilarStringsResult
)
from transformer_experiments.experiments.logit_lens import LogitLens

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ts = TinyShakespeareDataSet(cache_file='../artifacts/input.txt')
m, tokenizer = create_model_and_tokenizer(
    saved_model_filename='../artifacts/shakespeare-20231109.pt',
    dataset=ts,
    device=device,
)
_, val_data = split_text_dataset(ts.text, tokenizer, train_pct=0.9, device=device)
encoding_helpers = EncodingHelpers(tokenizer, device)
accessors = TransformerAccessors(m, device)

In [ ]:
print(f"device is {device}")

device is cpu


In [ ]:
if list(Path('../artifacts/block_internals_results/large_files/slen10/').glob('*')) == []:
    print("Run `make block_internals_slen10_dataset` in the project root to generate the required dataset")

In [ ]:
strings10 = all_unique_substrings(ts.text, 10)

In [ ]:
exp10 = BatchedBlockInternalsExperiment(
    eh=encoding_helpers,
    accessors=accessors,
    strings=strings10,
    output_dir=Path('../artifacts/block_internals_results/large_files/slen10/'),
    batch_size=10000,
)

In [ ]:
torch.manual_seed(1337)
n_samples = 20000
indices = torch.randperm(len(strings10))[:n_samples]
strings20k = [strings10[i.item()] for i in indices]

In [ ]:
# Create a sample of 500 strings
sample_size = 500
strings_sample = strings20k[:sample_size]

In [ ]:
# TODO: put this in a common component
def get_model_outputs(prompts: Sequence[str], encoding_helpers: EncodingHelpers):
    # Compute the model's predictions:
    tokens = encoding_helpers.tokenize_strings(prompts)
    logits, _ = m(tokens)

    logits = LogitsWrapper(logits, encoding_helpers.tokenizer)
    return [topk_tokens[-1] for topk_tokens in logits.topk_tokens(k=10)]


In [ ]:
model_outputs_sample = get_model_outputs(strings_sample, encoding_helpers)

In [ ]:
prompts_exp = BlockInternalsExperiment(encoding_helpers, accessors, strings_sample)

Start by examining what we get when we ask for a much larger top k values. 

In [ ]:
emb_sims, emb_distances = exp10.strings_with_topk_closest_embeddings(
    prompts_exp.embeddings[:5, :, :],
    k=200,
    largest=True,
    distance_function=batch_cosine_sim,
)

In [ ]:
emb_distances[:10, :], emb_distances[-10:, :]

(tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [0.9062, 0.9470, 0.9045, 0.9493, 0.9544],
         [0.9062, 0.9462, 0.8602, 0.9083, 0.9443],
         [0.9053, 0.9429, 0.8593, 0.9082, 0.9064],
         [0.9037, 0.9429, 0.8586, 0.9055, 0.9057],
         [0.9027, 0.9404, 0.8566, 0.9052, 0.9045],
         [0.9020, 0.9027, 0.8562, 0.9044, 0.8996],
         [0.9017, 0.9009, 0.8548, 0.9035, 0.8982],
         [0.8962, 0.9009, 0.8545, 0.9034, 0.8656],
         [0.8651, 0.9008, 0.8532, 0.9032, 0.8631]]),
 tensor([[0.7591, 0.7566, 0.7604, 0.8064, 0.8047],
         [0.7591, 0.7566, 0.7604, 0.8063, 0.8047],
         [0.7591, 0.7566, 0.7603, 0.8063, 0.8042],
         [0.7590, 0.7564, 0.7602, 0.8063, 0.8041],
         [0.7589, 0.7563, 0.7601, 0.8062, 0.8041],
         [0.7589, 0.7563, 0.7601, 0.8061, 0.8040],
         [0.7588, 0.7557, 0.7601, 0.8061, 0.8039],
         [0.7587, 0.7557, 0.7600, 0.8060, 0.8039],
         [0.7587, 0.7557, 0.7599, 0.8060, 0.8038],
         [0.7587, 0.7557, 0.7

In [ ]:
for j in range(10):
    print(f"{'   '.join([repr(emb_sims[i][j]) for i in range(len(emb_sims))])}")

print()
for j in range(-10, 0):
    print(f"{'   '.join([repr(emb_sims[i][j]) for i in range(len(emb_sims))])}")


'is dreams,'   'by present'   's eyes may'   'eart of ho'   ' man, as I'
'is dream o'   'My present'   's eye, mak'   'eart of mo'   ' men, as I'
'ur dreams,'   'be present'   's eyes in '   'earn of hi'   ' man, as y'
'of dreams,'   'dy present'   'l eyes can'   'ears of ha'   ' man, if I'
'us dreams.'   'my present'   's eyes to '   'park of ho'   'oman, as t'
'he dreams,'   'ry present'   'l eyes gaz'   'eart of ge'   ' men, as i'
'ly dreams,'   'y, present'   'r foes may'   'eard of hi'   ' many as y'
'en dreams,'   'on present'   'r ever may'   'east of yo'   'cian, as I'
'nd dreams,'   't, present'   's eyes do '   'part of hi'   ' son, as t'
'as dream\nS'   'in present'   's eye; tal'   'earn of yo'   ' long as I'

'is presenc'   ' a prisone'   'g over mas'   'efit of se'   ' man: we s'
'is prowess'   'ot prone t'   'l even tak'   'wist of ro'   ' wind as s'
'is be all,'   'is project'   't so I may'   'e is of so'   ' man; all '
'is the mad'   'my person.'   'n thou may'   'gen

In [ ]:
block_idx = 0
proj_sims, proj_distances = exp10.strings_with_topk_closest_proj_outputs(
    block_idx=block_idx,
    t_i=-1,
    queries=prompts_exp.proj_output(block_idx=block_idx)[:5, -1, :],
    k=200,
    largest=True,
    distance_function=batch_cosine_sim,
)

In [ ]:
proj_distances[:10, :], proj_distances[-10:, :]

(tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [0.9948, 0.9967, 0.9947, 0.9974, 0.9956],
         [0.9942, 0.9967, 0.9945, 0.9960, 0.9933],
         [0.9937, 0.9964, 0.9921, 0.9958, 0.9897],
         [0.9935, 0.9963, 0.9917, 0.9957, 0.9896],
         [0.9928, 0.9962, 0.9910, 0.9956, 0.9871],
         [0.9922, 0.9950, 0.9905, 0.9955, 0.9857],
         [0.9911, 0.9944, 0.9900, 0.9954, 0.9856],
         [0.9897, 0.9942, 0.9895, 0.9950, 0.9850],
         [0.9891, 0.9939, 0.9894, 0.9943, 0.9846]]),
 tensor([[0.9709, 0.9826, 0.9796, 0.9855, 0.9726],
         [0.9709, 0.9825, 0.9795, 0.9855, 0.9726],
         [0.9709, 0.9824, 0.9794, 0.9854, 0.9726],
         [0.9708, 0.9824, 0.9793, 0.9854, 0.9725],
         [0.9708, 0.9824, 0.9793, 0.9854, 0.9725],
         [0.9708, 0.9823, 0.9793, 0.9854, 0.9724],
         [0.9708, 0.9823, 0.9791, 0.9854, 0.9724],
         [0.9708, 0.9823, 0.9791, 0.9853, 0.9724],
         [0.9707, 0.9823, 0.9791, 0.9853, 0.9723],
         [0.9707, 0.9823, 0.9

In [ ]:
for j in range(10):
    print(f"{'   '.join([repr(proj_sims[i][j]) for i in range(len(proj_sims))])}")

print()
for j in range(-10, 0):
    print(f"{'   '.join([repr(proj_sims[i][j]) for i in range(len(proj_sims))])}")


'is dreams,'   'by present'   's eyes may'   'eart of ho'   ' man, as I'
'ly dreams,'   'my present'   'e case may'   'ster of ho'   ' men, as I'
'en dreams,'   'dy present'   ' sense may'   'ffer to ha'   ' and, as I'
'he dreams,'   'ry present'   'ied as may'   'ruth of ho'   ' not, as I'
'ur dreams,'   'be present'   'ay she may'   'otes of ho'   'o me, as I'
'nd dreams,'   'My present'   'r foes may'   'anes of ho'   'nd I, as I'
'ery beams,'   'y; present'   'So she may'   'oint of ho'   'cian, as I'
'of dreams,'   'in present'   ' haste may'   'yers of ho'   'I am, as t'
"n's beams,"   'is present'   'odesty may'   'ains of ho'   '-day, as I'
'hese arms,'   'im present'   'esence may'   'ound of ho'   '\nAnd, as I'

'sires most'   'ast ungent'   "'s some am"   'lf with ho'   ' be, was l'
'teous mass'   'What scene'   'e they mad'   'tell of hi'   'Look, as I'
'm to kiss,'   'lest scent'   ' am to say'   'Thus to ha'   'wick, as o'
'much amiss'   'ondon sent'   ' early mad'   's o

In [ ]:
block_idx = 0
ffwd_sims, ffwd_distances = exp10.strings_with_topk_closest_ffwd_outputs(
    block_idx=block_idx,
    t_i=-1,
    queries=prompts_exp.ffwd_output(block_idx=block_idx)[:5, -1, :],
    k=200,
    largest=True,
    distance_function=batch_cosine_sim,
)

In [ ]:
ffwd_distances[:10, :], ffwd_distances[-10:, :]

(tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [0.9998, 0.9999, 0.9997, 0.9998, 0.9999],
         [0.9998, 0.9998, 0.9996, 0.9998, 0.9997],
         [0.9998, 0.9998, 0.9996, 0.9998, 0.9997],
         [0.9998, 0.9998, 0.9995, 0.9998, 0.9997],
         [0.9997, 0.9998, 0.9995, 0.9998, 0.9996],
         [0.9997, 0.9997, 0.9995, 0.9998, 0.9995],
         [0.9997, 0.9997, 0.9995, 0.9998, 0.9995],
         [0.9996, 0.9997, 0.9995, 0.9997, 0.9994],
         [0.9996, 0.9997, 0.9995, 0.9997, 0.9994]]),
 tensor([[0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9

In [ ]:
for j in range(10):
    print(f"{'   '.join([repr(ffwd_sims[i][j]) for i in range(len(ffwd_sims))])}")

print()
for j in range(-10, 0):
    print(f"{'   '.join([repr(ffwd_sims[i][j]) for i in range(len(ffwd_sims))])}")

'is dreams,'   'by present'   's eyes may'   'eart of ho'   ' man, as I'
'en dreams,'   'my present'   ' sense may'   'ster of ho'   ' men, as I'
'ly dreams,'   'dy present'   'e case may'   'otes of ho'   ' and, as I'
'he dreams,'   'ry present'   'r foes may'   'anes of ho'   ' not, as I'
'nd dreams,'   'My present'   'ied as may'   'ains of ho'   'o me, as I'
'ur dreams,'   'be present'   'ay she may'   'oint of ho'   'nd I, as I'
'of dreams,'   'y, present'   ' bones may'   'ound of ho'   '-day, as I'
'ery beams,'   'y; present'   'So she may'   'fear to ho'   ' but, as I'
'rate arms,'   'im present'   ' grace may'   'ally of ho'   'rd me as I'
'hese arms,'   'in present'   'e more may'   'yers of ho'   '\nYes, as I'

'ck groans,'   'll\nPresent'   'en you say'   'ace our ho'   'r sakes, I'
'te builds,'   's innocent'   'Hope I may'   'he that ho'   'early as I'
'she finds,'   'me Florent'   'e must say'   'Half an ho'   'nes, and I'
'reat loss,'   'their gent'   'n thou may'   'as

## For the 500 Sample Strings, Can we Calculate Cosine Similarity to Every Other String?


In [ ]:
class CosineSimilaritiesExperiment:
    def __init__(
        self,
        exp: BatchedBlockInternalsExperiment,
        output_folder: Path,
    ):
        self.exp = exp
        self.output_folder = output_folder

        self.n_batches = exp.n_batches

    def embedding_sims_filename(self, batch_idx: int):
        return self.output_folder / f'embedding_sims_{batch_idx:03d}.pt'

    def proj_out_sims_filename(self, batch_idx: int, block_idx: int):
        return self.output_folder / f'proj_out_sims_{batch_idx:03d}_{block_idx:02d}.pt'

    def ffwd_out_sims_filename(self, batch_idx: int, block_idx: int):
        return self.output_folder / f'ffwds_out_sims_{batch_idx:03d}_{block_idx:02d}.pt'

    def generate_embedding_sims(self, chunk_size: int, queries: torch.Tensor, disable_progress_bar: bool = False):
        n_chunks = exp10.batch_size // chunk_size

        assert queries.dim() == 2
        n_queries = queries.shape[0]

        def sims_for_chunk(emb_batch: torch.Tensor, chunk_idx: int):
            chunk = emb_batch[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, :, :]
            actual_chunk_size = chunk.shape[0]
            return F.cosine_similarity(
                chunk.reshape(actual_chunk_size, 1, -1).expand(-1, n_queries, -1),
                queries,
                dim=-1
            )

        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            emb_batch = torch.load(str(self.exp._embeddings_filename(batch_idx=batch_idx)), mmap=True)
            sims = torch.cat([
                sims_for_chunk(emb_batch, i)
                for i in range(n_chunks)
            ], dim=0)
            torch.save(sims, str(self.embedding_sims_filename(batch_idx=batch_idx)))

    def generate_proj_out_sims(self, get_queries: Callable[[int], torch.Tensor], disable_progress_bar=False):
        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            for block_idx in range(n_layer):
                queries = get_queries(block_idx)

                assert queries.dim() == 2
                n_queries = queries.shape[0]

                proj_out_batch = torch.load(str(self.exp._proj_output_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True)
                batch_size = proj_out_batch.shape[0]
                sims = F.cosine_similarity(
                    proj_out_batch[:, -1, :].reshape(batch_size, 1, -1).expand(-1, n_queries, -1),
                    queries,
                    dim=-1
                )
                torch.save(sims, str(self.proj_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)))

    def generate_ffwd_out_sims(self, get_queries: Callable[[int], torch.Tensor], disable_progress_bar=False):
        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            for block_idx in range(n_layer):
                queries = get_queries(block_idx)
                assert queries.dim() == 2
                n_queries = queries.shape[0]

                ffwd_out_batch = torch.load(str(self.exp._ffwd_output_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True)
                batch_size = ffwd_out_batch.shape[0]
                sims = F.cosine_similarity(
                    ffwd_out_batch[:, -1, :].reshape(batch_size, 1, -1).expand(-1, n_queries, -1),
                    queries,
                    dim=-1
                )
                torch.save(sims, str(self.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)))



In [ ]:
# Start by doing some analysis on the first 500 strings.
output_folder = exp10.output_dir / 'cosine_sims_000'
output_folder.mkdir(exist_ok=True)

In [ ]:
cos_exp = CosineSimilaritiesExperiment(exp10, output_folder)

In [ ]:
cos_exp.generate_embedding_sims(chunk_size=2000, queries=prompts_exp.embeddings.reshape(sample_size, -1))

In [ ]:
cos_exp.generate_proj_out_sims(get_queries=lambda block_idx: prompts_exp.proj_output(block_idx=block_idx)[:, -1, :])

In [ ]:
cos_exp.generate_ffwd_out_sims(get_queries=lambda block_idx: prompts_exp.ffwd_output(block_idx=block_idx)[:, -1, :])

In [ ]:
# Now generate cosine sims for ffwd output for the remaining 15000 strings
n_all_strings = 20000
sample_size = 500
n_chunks = math.ceil(n_all_strings / sample_size)

for sample_idx in tqdm(range(0, n_chunks)):
    start_idx = sample_idx * sample_size
    end_idx = min(start_idx + sample_size, n_all_strings)

    strings = strings20k[start_idx:end_idx]

    output_folder = exp10.output_dir / f'cosine_sims_{sample_idx:03d}'
    output_folder.mkdir(exist_ok=True)

    exp = CosineSimilaritiesExperiment(exp10, output_folder)

    sample_prompts_exp = BlockInternalsExperiment(encoding_helpers, accessors, strings)

    exp.generate_ffwd_out_sims(
        get_queries=lambda block_idx: sample_prompts_exp.ffwd_output(block_idx=block_idx)[:, -1, :],
        disable_progress_bar=True,
    )

## Analyze the Results

First, collect some stats about the results

In [ ]:
def stats_across_batches(
    get_batch: Callable[[int], torch.Tensor],
    n_batches: int,
    n_queries: int,
):
    mins = torch.zeros(n_queries)
    maxs = torch.zeros(n_queries)
    means = torch.zeros(n_queries)
    s = torch.zeros(n_queries)

    total_count = 0
    for i in range(n_batches):
        batch = get_batch(i)
        batch_size, n_queries_batch = batch.shape
        assert n_queries_batch == n_queries

        mins = torch.minimum(mins, batch.min(dim=0).values)
        maxs = torch.maximum(maxs, batch.max(dim=0).values)

        # Implement Chan, Golub, and LeVeque method
        total_count += batch_size
        delta = batch.mean(dim=0) - means
        means += delta * batch_size / total_count
        s += batch.var(dim=0) * (batch_size - 1) + delta**2 * batch_size * (total_count - batch_size) / total_count

    vars = s / (total_count - 1)
    stds = torch.sqrt(vars)
    return mins, maxs, means, stds

In [ ]:
ffwd_mins, ffwd_maxs, ffwd_means, ffwd_stds = zip(*[
    stats_across_batches(
        get_batch=lambda i: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=i, block_idx=block_idx))),
        n_batches=cos_exp.n_batches,
        n_queries=sample_size,
    )
    for block_idx in range(n_layer)
])

In [ ]:
for block_idx in range(n_layer):
    print(f"Block {block_idx}: ffwd_out mean: {ffwd_means[block_idx].mean():.3f} ± {ffwd_means[block_idx].std():.3f}")

Block 0: ffwd_out mean: 0.165 ± 0.029
Block 1: ffwd_out mean: 0.095 ± 0.030
Block 2: ffwd_out mean: 0.053 ± 0.024
Block 3: ffwd_out mean: 0.057 ± 0.028
Block 4: ffwd_out mean: 0.069 ± 0.047
Block 5: ffwd_out mean: 0.090 ± 0.069


In [ ]:
for block_idx in range(n_layer):
    print(f"Block {block_idx}: ffwd_out mean: {ffwd_means[block_idx].mean():.3f} ± {ffwd_means[block_idx].std():.3f}")

Block 0: ffwd_out mean: 0.801 ± 0.092
Block 1: ffwd_out mean: 0.365 ± 0.042
Block 2: ffwd_out mean: 0.160 ± 0.033
Block 3: ffwd_out mean: 0.072 ± 0.036
Block 4: ffwd_out mean: 0.088 ± 0.059
Block 5: ffwd_out mean: 0.133 ± 0.091


In [ ]:
def filter_across_batches(
    get_batch: Callable[[int], torch.Tensor],
    n_batches: int,
    filter_fn: Callable[[torch.Tensor], torch.Tensor],
    n_queries: int,
):
    total_count = 0
    matching_indices = [[] for _ in range(n_queries)]
    for i in range(n_batches):
        batch = get_batch(i)
        batch_size, n_queries_batch = batch.shape
        assert n_queries_batch == n_queries

        filtered = filter_fn(batch)
        nonzeros = torch.nonzero(filtered)
        for i in range(nonzeros.shape[0]):
            idx_in_batch, query_idx = nonzeros[i, :]
            matching_indices[query_idx.item()].append(total_count + idx_in_batch.item())

        total_count += batch_size

    return matching_indices

In [ ]:
# Tests for filter_across_batches()

batches = [
    torch.tensor([
        [0.0, 0.6, 0.4, 0.3],
        [0.1, 0.3, 0.5, 0.1],
        [0.0, 0.1, 0.8, 0.0],
    ]),
    torch.tensor([
        [0.7, 0.2, 0.6, 0.3],
        [0.1, 0.8, 0.2, 0.8],
    ]),
]

result = filter_across_batches(
    get_batch=lambda i: batches[i],
    n_batches=len(batches),
    filter_fn=lambda batch: batch > 0.5,
    n_queries=4,
)
test_eq(result, [
    [3,],
    [0, 4],
    [2, 3],
    [4],
])

In [ ]:
def filter_result_stats(
    filter_results: List[List[int]],
):
    lens = [len(result) for result in filter_results]
    return {
        'min': min(lens),
        'max': max(lens),
        'mean': np.mean(lens),
        'std': np.std(lens),
    }

In [ ]:
def get_matching_strings(
    filter_result: List[List[int]],
    strings: Sequence[str],
):
    return [
        [
            strings[j]
            for j in filter_result[i]
        ]
        for i in range(len(filter_result))
    ]

In [ ]:
next_token_map10 = build_next_token_map(ts.text, 10, tokenizer.vocab_size, tokenizer.stoi)

In [ ]:
# TODO: put this in a common component
def analyze_simulate_results(sim_freqs, model_outputs):
    assert len(sim_freqs) == len(model_outputs)
    topn_matches = [0 for _ in range(10)]
    topn_matches_any_order = [0 for _ in range(10)]
    for i, sim_freq in enumerate(sim_freqs):
        sim_output = top_nonzero_tokens(sim_freq, encoding_helpers.tokenizer.itos)[:10]
        model_output = model_outputs[i]

        sim_tokens, _ = zip(*sim_output)
        model_tokens, _ = zip(*model_output)

        n = min(len(sim_tokens), len(model_tokens))
        for j in range(n):
            if sim_tokens[j] == model_tokens[j]:
                topn_matches[j] += 1
            if set(sim_tokens[:j+1]) == set(model_tokens[:j+1]):
                topn_matches_any_order[j] += 1

    return topn_matches, topn_matches_any_order


In [ ]:
def analyze_dataset(
    get_batch: Callable[[int], torch.Tensor],
    n_batches: int,
    filter_fn: Callable[[torch.Tensor], torch.Tensor],
    n_queries: int,
    all_strings: Sequence[str],
    next_token_map: Dict[str, torch.Tensor],
    model_outputs: Sequence[Sequence[Tuple[str, float]]],
):
    filter_results = filter_across_batches(
        get_batch=get_batch,
        n_batches=n_batches,
        filter_fn=filter_fn,
        n_queries=n_queries,
    )

    print(filter_result_stats(filter_results))

    filter_results_strings = get_matching_strings(filter_results, all_strings)
    filter_result_freqs = [
        torch.stack([
            next_token_map[matching_string]
            for matching_string in matching_strings
        ]).sum(dim=0)
        for matching_strings in filter_results_strings
    ]

    filter_result_probs = [
        freqs / freqs.sum()
        for freqs in filter_result_freqs
    ]

    topn_matches, topn_matches_any_order = analyze_simulate_results(filter_result_probs, model_outputs)
    for i in range(10):
        print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
        print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

    return filter_result_freqs

In [ ]:
block_idx = 5
ffwd5_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.86,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 57024, 'mean': 5336.108, 'std': 11856.829263354348}
Top 1 matches: 0.786
Top 1 matches (any order): 0.786
Top 2 matches: 0.448
Top 2 matches (any order): 0.486
Top 3 matches: 0.282
Top 3 matches (any order): 0.338
Top 4 matches: 0.188
Top 4 matches (any order): 0.264
Top 5 matches: 0.166
Top 5 matches (any order): 0.192
Top 6 matches: 0.142
Top 6 matches (any order): 0.178
Top 7 matches: 0.116
Top 7 matches (any order): 0.128
Top 8 matches: 0.108
Top 8 matches (any order): 0.102
Top 9 matches: 0.062
Top 9 matches (any order): 0.086
Top 10 matches: 0.052
Top 10 matches (any order): 0.044


In [ ]:
block_idx = 4
ffwd4_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.82,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 12997, 'mean': 696.508, 'std': 1755.2617850155573}
Top 1 matches: 0.800
Top 1 matches (any order): 0.800
Top 2 matches: 0.440
Top 2 matches (any order): 0.494
Top 3 matches: 0.272
Top 3 matches (any order): 0.338
Top 4 matches: 0.194
Top 4 matches (any order): 0.236
Top 5 matches: 0.162
Top 5 matches (any order): 0.182
Top 6 matches: 0.090
Top 6 matches (any order): 0.130
Top 7 matches: 0.074
Top 7 matches (any order): 0.078
Top 8 matches: 0.070
Top 8 matches (any order): 0.064
Top 9 matches: 0.054
Top 9 matches (any order): 0.062
Top 10 matches: 0.048
Top 10 matches (any order): 0.054


In [ ]:
block_idx = 3
ffwd3_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.80,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 7598, 'mean': 473.594, 'std': 1144.8101122736468}
Top 1 matches: 0.812
Top 1 matches (any order): 0.812
Top 2 matches: 0.470
Top 2 matches (any order): 0.512
Top 3 matches: 0.280
Top 3 matches (any order): 0.366
Top 4 matches: 0.206
Top 4 matches (any order): 0.246
Top 5 matches: 0.160
Top 5 matches (any order): 0.194
Top 6 matches: 0.130
Top 6 matches (any order): 0.136
Top 7 matches: 0.088
Top 7 matches (any order): 0.080
Top 8 matches: 0.062
Top 8 matches (any order): 0.064
Top 9 matches: 0.066
Top 9 matches (any order): 0.076
Top 10 matches: 0.056
Top 10 matches (any order): 0.064


In [ ]:
block_idx = 2
ffwd2_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.92,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 5826, 'mean': 246.946, 'std': 670.6073121313248}
Top 1 matches: 0.810
Top 1 matches (any order): 0.810
Top 2 matches: 0.466
Top 2 matches (any order): 0.500
Top 3 matches: 0.230
Top 3 matches (any order): 0.276
Top 4 matches: 0.174
Top 4 matches (any order): 0.180
Top 5 matches: 0.126
Top 5 matches (any order): 0.164
Top 6 matches: 0.100
Top 6 matches (any order): 0.094
Top 7 matches: 0.044
Top 7 matches (any order): 0.058
Top 8 matches: 0.078
Top 8 matches (any order): 0.052
Top 9 matches: 0.032
Top 9 matches (any order): 0.056
Top 10 matches: 0.046
Top 10 matches (any order): 0.042


In [ ]:
block_idx = 1
ffwd1_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.95,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 5094, 'mean': 475.298, 'std': 823.7842880244804}
Top 1 matches: 0.758
Top 1 matches (any order): 0.758
Top 2 matches: 0.428
Top 2 matches (any order): 0.474
Top 3 matches: 0.256
Top 3 matches (any order): 0.358
Top 4 matches: 0.192
Top 4 matches (any order): 0.230
Top 5 matches: 0.180
Top 5 matches (any order): 0.200
Top 6 matches: 0.116
Top 6 matches (any order): 0.142
Top 7 matches: 0.098
Top 7 matches (any order): 0.104
Top 8 matches: 0.076
Top 8 matches (any order): 0.078
Top 9 matches: 0.064
Top 9 matches (any order): 0.076
Top 10 matches: 0.062
Top 10 matches (any order): 0.042


In [ ]:
block_idx = 0
ffwd0_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.95,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 5174, 'mean': 476.89, 'std': 769.571469000768}
Top 1 matches: 0.734
Top 1 matches (any order): 0.734
Top 2 matches: 0.428
Top 2 matches (any order): 0.466
Top 3 matches: 0.262
Top 3 matches (any order): 0.340
Top 4 matches: 0.200
Top 4 matches (any order): 0.230
Top 5 matches: 0.164
Top 5 matches (any order): 0.190
Top 6 matches: 0.122
Top 6 matches (any order): 0.144
Top 7 matches: 0.096
Top 7 matches (any order): 0.106
Top 8 matches: 0.078
Top 8 matches (any order): 0.078
Top 9 matches: 0.052
Top 9 matches (any order): 0.078
Top 10 matches: 0.054
Top 10 matches (any order): 0.048


In [ ]:
total_freqs = [
    (
        0.005*ffwd0_freqs[i] +
        0.01*ffwd1_freqs[i] +
        0.1*ffwd2_freqs[i] +
        0.1*ffwd3_freqs[i] +
        3*ffwd4_freqs[i] +
        0.005*ffwd5_freq
    )
    for i, ffwd5_freq in enumerate(ffwd5_freqs)
]
total_probs = [
    freqs / freqs.sum()
    for freqs in total_freqs
]
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs_sample)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

Top 1 matches: 0.814
Top 1 matches (any order): 0.814
Top 2 matches: 0.514
Top 2 matches (any order): 0.562
Top 3 matches: 0.348
Top 3 matches (any order): 0.402
Top 4 matches: 0.250
Top 4 matches (any order): 0.312
Top 5 matches: 0.198
Top 5 matches (any order): 0.234
Top 6 matches: 0.160
Top 6 matches (any order): 0.192
Top 7 matches: 0.126
Top 7 matches (any order): 0.136
Top 8 matches: 0.118
Top 8 matches (any order): 0.116
Top 9 matches: 0.080
Top 9 matches (any order): 0.096
Top 10 matches: 0.082
Top 10 matches (any order): 0.084


## Incorporate data from the rest of the strings (everything above just looked at first 500)

In [ ]:
n_all_strings = 20000
sample_size = 500
n_chunks = math.ceil(n_all_strings / sample_size)

next_token_map = next_token_map10
all_strings = strings10

ffwd_thresholds = [0.95, 0.95, 0.92, 0.80, 0.82, 0.86]
ffwd_freqs = [[] for _ in range(n_layer)]
model_outputs = []

for sample_idx in tqdm(range(0, n_chunks)):
    output_folder = exp10.output_dir / f"cosine_sims_{sample_idx:03d}"
    output_folder.mkdir(exist_ok=True)

    exp = CosineSimilaritiesExperiment(exp10, output_folder)

    start_idx = sample_idx * sample_size
    end_idx = min(start_idx + sample_size, n_all_strings)

    strings = strings20k[start_idx:end_idx]

    model_outputs.extend(get_model_outputs(strings, encoding_helpers))

    for block_idx in range(n_layer):
        get_batch = lambda batch_idx: torch.load(
            str(exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)),
            mmap=True,
        )

        filter_results = filter_across_batches(
            get_batch=get_batch,
            n_batches=exp.n_batches,
            filter_fn=lambda batch: batch > ffwd_thresholds[block_idx],
            n_queries=sample_size,
        )
        filter_results_strings = get_matching_strings(filter_results, all_strings)
        filter_result_freqs = [
            torch.stack(
                [
                    next_token_map[matching_string]
                    for matching_string in matching_strings
                ]
            ).sum(dim=0)
            for matching_strings in filter_results_strings
        ]
        ffwd_freqs[block_idx].extend(filter_result_freqs)

ffwd_freqs = torch.stack(
    [torch.stack(ffwd_freqs[block_idx]) for block_idx in range(n_layer)]
)

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
ffwd_freqs.shape, len(model_outputs)

(torch.Size([6, 20000, 65]), 20000)

In [ ]:
(exp10.output_dir / 'learn_coefficients').mkdir(exist_ok=True)
torch.save(ffwd_freqs, str(exp10.output_dir / 'learn_coefficients/ffwd_freqs.pt'))

In [ ]:
# Check that we still get the same results for the first 500
hand_rolled_coeffs = torch.tensor([0.005, 0.01, 0.1, 0.1, 3, 0.005]) .unsqueeze(dim=1).unsqueeze(dim=2) # (n_layer, 1, 1)
total_freqs = (ffwd_freqs[:, :500, :] * hand_rolled_coeffs).sum(dim=0)
total_probs = total_freqs / total_freqs.sum(dim=-1, keepdim=True)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs[:500])
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

Top 1 matches: 0.814
Top 1 matches (any order): 0.814
Top 2 matches: 0.514
Top 2 matches (any order): 0.562
Top 3 matches: 0.348
Top 3 matches (any order): 0.402
Top 4 matches: 0.250
Top 4 matches (any order): 0.312
Top 5 matches: 0.198
Top 5 matches (any order): 0.234
Top 6 matches: 0.160
Top 6 matches (any order): 0.192
Top 7 matches: 0.126
Top 7 matches (any order): 0.136
Top 8 matches: 0.118
Top 8 matches (any order): 0.116
Top 9 matches: 0.080
Top 9 matches (any order): 0.096
Top 10 matches: 0.082
Top 10 matches (any order): 0.084


In [ ]:
# Look at it for all samples
hand_rolled_coeffs = torch.tensor([0.005, 0.01, 0.1, 0.1, 3, 0.005]) .unsqueeze(dim=1).unsqueeze(dim=2) # (n_layer, 1, 1)
total_freqs = (ffwd_freqs * hand_rolled_coeffs).sum(dim=0)
total_probs = total_freqs / total_freqs.sum(dim=-1, keepdim=True)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.814
Top 1 matches (any order): 0.814
Top 2 matches: 0.498
Top 2 matches (any order): 0.556
Top 3 matches: 0.341
Top 3 matches (any order): 0.412
Top 4 matches: 0.271
Top 4 matches (any order): 0.339
Top 5 matches: 0.209
Top 5 matches (any order): 0.257
Top 6 matches: 0.178
Top 6 matches (any order): 0.211
Top 7 matches: 0.132
Top 7 matches (any order): 0.153
Top 8 matches: 0.112
Top 8 matches (any order): 0.129
Top 9 matches: 0.097
Top 9 matches (any order): 0.107
Top 10 matches: 0.063
Top 10 matches (any order): 0.066


But can we tweak these hand-rolled coefficients for the full data set and get better results?

In [ ]:
hand_rolled_coeffs = torch.tensor([0.001, 0.01, 0.01, 0.9, 3, 0.05]) .unsqueeze(dim=1).unsqueeze(dim=2) # (n_layer, 1, 1)
total_freqs = (ffwd_freqs * hand_rolled_coeffs).sum(dim=0)
total_probs = total_freqs / total_freqs.sum(dim=-1, keepdim=True)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.814
Top 1 matches (any order): 0.814
Top 2 matches: 0.500
Top 2 matches (any order): 0.562
Top 3 matches: 0.346
Top 3 matches (any order): 0.420
Top 4 matches: 0.277
Top 4 matches (any order): 0.345
Top 5 matches: 0.210
Top 5 matches (any order): 0.259
Top 6 matches: 0.177
Top 6 matches (any order): 0.210
Top 7 matches: 0.133
Top 7 matches (any order): 0.154
Top 8 matches: 0.112
Top 8 matches (any order): 0.129
Top 9 matches: 0.099
Top 9 matches (any order): 0.109
Top 10 matches: 0.064
Top 10 matches (any order): 0.068


This is the best yet for the full data set, with hand-rolled coefficients. Let's see if we can learn better values. 

## Learn Coefficients

In [ ]:
class ModelSim(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.coeffs = torch.nn.Parameter(
            torch.randn(n_layer, 1, 1, dtype=torch.float32, requires_grad=True)
        )
        torch.nn.init.normal_(self.coeffs.data, mean=0.0, std=0.2)

    def forward(self, freqs: torch.Tensor, model_output: Optional[torch.Tensor]=None):
        total_freqs = (freqs * self.coeffs).sum(dim=0)
        total_probs = total_freqs / total_freqs.sum(dim=-1, keepdim=True)

        if model_output is not None:
            loss = torch.norm(total_probs - model_output, dim=-1).sum()
        else:
            loss = None


        return total_probs, loss

In [ ]:
@torch.no_grad()
def estimate_loss(
    model: ModelSim, get_batch_func: GetBatchFunction, eval_iters: int=100
):
    out = {}
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch_func(split=split)

            _, loss = model(X, Y)

            losses[k] = loss.item()

        out[split] = losses.mean()
    return out

In [ ]:
def get_batch(batch_size: int, freqs: torch.Tensor, split: str='train', train_pct: float=0.9):
    n = freqs.shape[1]
    assert split in ['train', 'val']
    n_train = int(n * train_pct)
    low = 0 if split == 'train' else n_train
    high = n_train if split == 'train' else n

    batch_indices = torch.randint(low=low, high=high, size=(batch_size,), dtype=torch.long)
    batch_strings = [strings20k[i.item()] for i in batch_indices]

    tokens = encoding_helpers.tokenize_strings(batch_strings)
    logits, _ = m(tokens)
    model_output = F.softmax(logits[:, -1, :], dim=-1)

    return freqs[:, batch_indices, :].clone(), model_output.detach()


In [ ]:
batch_size=500
eval_iters = 100

get_batch_func = partial(get_batch, batch_size=batch_size, freqs=ffwd_freqs, train_pct=0.9)
estimate_loss_func = partial(estimate_loss, get_batch_func=get_batch_func, eval_iters=eval_iters)

In [ ]:
_ = torch.manual_seed(1337) # Ensure stable random values
m2 = ModelSim()
_ = m2.to(device)

In [ ]:
output_dir = exp10.output_dir / 'learn_coefficients'
output_dir.mkdir(exist_ok=True)
checkpointer = CheckPointer(output_dir, 'coeff_model_checkpoint', start_num=0)

In [ ]:
trainer = Trainer(
    model=m2,
    checkpointer=checkpointer,
    get_batch_func=get_batch_func,
    estimate_loss_func=estimate_loss_func,
    iters_trained=0
)

In [ ]:
# Get a starting point for the loss
estimate_loss_func(m2)

{'train': tensor(397.8691), 'val': tensor(419.5697)}

In [ ]:
# Start with a pretty high learning rate and go for 1000 iterations
learning_rate = 3e-2
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

trainer.train(1000, optimizer, eval_interval=500)

  0%|          | 0/1000 [00:00<?, ?it/s]

step 499: train loss 100.8659, val loss 100.7756
step 999: train loss 98.1379, val loss 98.4768


In [ ]:
m2.coeffs.data

tensor([[[0.0683]],

        [[0.1038]],

        [[0.1724]],

        [[0.0944]],

        [[0.2360]],

        [[0.1929]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.768
Top 1 matches (any order): 0.768
Top 2 matches: 0.458
Top 2 matches (any order): 0.520
Top 3 matches: 0.316
Top 3 matches (any order): 0.381
Top 4 matches: 0.256
Top 4 matches (any order): 0.320
Top 5 matches: 0.195
Top 5 matches (any order): 0.235
Top 6 matches: 0.169
Top 6 matches (any order): 0.202
Top 7 matches: 0.131
Top 7 matches (any order): 0.149
Top 8 matches: 0.111
Top 8 matches (any order): 0.128
Top 9 matches: 0.093
Top 9 matches (any order): 0.102
Top 10 matches: 0.059
Top 10 matches (any order): 0.061


Not bad for 1 round of training but we know there is a better solution. Let's try a few more rounds at this learning rate.

In [ ]:
trainer.train(1000, optimizer, eval_interval=500)

  0%|          | 0/1000 [00:00<?, ?it/s]

step 499: train loss 86.7757, val loss 85.7378
step 999: train loss 86.2982, val loss 85.2880


In [ ]:
m2.coeffs.data

tensor([[[ 0.0016]],

        [[-0.0015]],

        [[ 0.1099]],

        [[ 0.0926]],

        [[ 0.2220]],

        [[ 0.1237]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.791
Top 1 matches (any order): 0.791
Top 2 matches: 0.479
Top 2 matches (any order): 0.541
Top 3 matches: 0.332
Top 3 matches (any order): 0.397
Top 4 matches: 0.264
Top 4 matches (any order): 0.329
Top 5 matches: 0.201
Top 5 matches (any order): 0.244
Top 6 matches: 0.171
Top 6 matches (any order): 0.207
Top 7 matches: 0.131
Top 7 matches (any order): 0.151
Top 8 matches: 0.110
Top 8 matches (any order): 0.127
Top 9 matches: 0.095
Top 9 matches (any order): 0.102
Top 10 matches: 0.059
Top 10 matches (any order): 0.061


It's still going down, so let's keep trying. Let it run more rounds and we can always backtrack if it overshoots.

In [ ]:
trainer.train(5000, optimizer, eval_interval=500)

  0%|          | 0/5000 [00:00<?, ?it/s]

step 499: train loss 83.4608, val loss 82.6918
step 999: train loss 101.8541, val loss 103.2045
step 1499: train loss 102.3397, val loss 102.5408
step 1999: train loss 102.3909, val loss 102.3771
step 2499: train loss 101.9758, val loss 102.9581
step 2999: train loss 102.1723, val loss 103.8465
step 3499: train loss 102.7493, val loss 102.4819
step 3999: train loss 101.7291, val loss 103.1296
step 4499: train loss 102.1053, val loss 103.0395
step 4999: train loss 101.2512, val loss 102.7032


OK, that clearly overshot. Let's go back to the good point and try a smaller learning rate.

In [ ]:
checkpoint = torch.load(checkpointer.output_dir / 'coeff_model_checkpoint_000004.pt')
checkpoint['iters'], checkpoint['train_loss'], checkpoint['val_loss']

(2500, tensor(83.4608), tensor(82.6918))

In [ ]:
m2.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

Now we are back at a good state. 

In [ ]:
m2.coeffs.data

tensor([[[ 0.0034]],

        [[-0.0006]],

        [[ 0.0434]],

        [[ 0.0864]],

        [[ 0.2299]],

        [[ 0.0290]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.803
Top 1 matches (any order): 0.803
Top 2 matches: 0.490
Top 2 matches (any order): 0.552
Top 3 matches: 0.340
Top 3 matches (any order): 0.410
Top 4 matches: 0.272
Top 4 matches (any order): 0.335
Top 5 matches: 0.205
Top 5 matches (any order): 0.254
Top 6 matches: 0.178
Top 6 matches (any order): 0.209
Top 7 matches: 0.133
Top 7 matches (any order): 0.154
Top 8 matches: 0.114
Top 8 matches (any order): 0.128
Top 9 matches: 0.095
Top 9 matches (any order): 0.103
Top 10 matches: 0.060
Top 10 matches (any order): 0.064


Not bad but we know we can do better.

In [ ]:
# Reduce the learning rate by one order of magnitude
learning_rate = 3e-3
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

# Train some more
trainer.train(2000, optimizer, eval_interval=500)

  0%|          | 0/2000 [00:00<?, ?it/s]

step 499: train loss 86.5964, val loss 85.8480
step 999: train loss 85.6861, val loss 86.0898
step 1499: train loss 86.2156, val loss 85.8741
step 1999: train loss 86.1549, val loss 86.1937


This got worse. Let's go back to the checkpoint and try an even smaller learning rate. 

In [ ]:
checkpoint['iters'], checkpoint['train_loss'], checkpoint['val_loss']

(2500, tensor(83.4608), tensor(82.6918))

In [ ]:
m2.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
# Go down one more order of magnitude
learning_rate = 3e-4
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

# Train some more
trainer.train(2000, optimizer, eval_interval=500)

  0%|          | 0/2000 [00:00<?, ?it/s]

step 499: train loss 77.9547, val loss 76.8849
step 999: train loss 81.7565, val loss 81.3437
step 1499: train loss 81.5345, val loss 81.1168
step 1999: train loss 81.3728, val loss 80.8595


That made some progress and then reverted. Let's go back to the checkpoint that was the best. 

In [ ]:
checkpoint = torch.load(checkpointer.output_dir / 'coeff_model_checkpoint_000018.pt')
checkpoint['iters'], checkpoint['train_loss'], checkpoint['val_loss']

(9500, tensor(77.9547), tensor(76.8849))

In [ ]:
m2.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
m2.coeffs.data

tensor([[[ 0.0018]],

        [[-0.0005]],

        [[-0.0024]],

        [[ 0.0622]],

        [[ 0.2803]],

        [[ 0.0006]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.814
Top 1 matches (any order): 0.814
Top 2 matches: 0.499
Top 2 matches (any order): 0.559
Top 3 matches: 0.340
Top 3 matches (any order): 0.414
Top 4 matches: 0.274
Top 4 matches (any order): 0.342
Top 5 matches: 0.209
Top 5 matches (any order): 0.258
Top 6 matches: 0.178
Top 6 matches (any order): 0.208
Top 7 matches: 0.130
Top 7 matches (any order): 0.151
Top 8 matches: 0.110
Top 8 matches (any order): 0.126
Top 9 matches: 0.095
Top 9 matches (any order): 0.106
Top 10 matches: 0.062
Top 10 matches (any order): 0.066


Still not as good as the hand-rolled coefficients but very close. Let's try one final time with a smaller learning rate.

In [ ]:
# Go down one more order of magnitude
learning_rate = 3e-5
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

# Train some more
trainer.train(2000, optimizer, eval_interval=500)

  0%|          | 0/2000 [00:00<?, ?it/s]

step 499: train loss 77.4044, val loss 77.3757
step 999: train loss 77.7672, val loss 76.8020
step 1499: train loss 77.8616, val loss 76.1973
step 1999: train loss 77.6942, val loss 76.3575


In [ ]:
m2.coeffs.data

tensor([[[ 0.0014]],

        [[-0.0006]],

        [[-0.0019]],

        [[ 0.0627]],

        [[ 0.2819]],

        [[ 0.0006]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.815
Top 1 matches (any order): 0.815
Top 2 matches: 0.499
Top 2 matches (any order): 0.560
Top 3 matches: 0.340
Top 3 matches (any order): 0.413
Top 4 matches: 0.274
Top 4 matches (any order): 0.341
Top 5 matches: 0.209
Top 5 matches (any order): 0.257
Top 6 matches: 0.177
Top 6 matches (any order): 0.207
Top 7 matches: 0.130
Top 7 matches (any order): 0.150
Top 8 matches: 0.109
Top 8 matches (any order): 0.125
Top 9 matches: 0.095
Top 9 matches (any order): 0.106
Top 10 matches: 0.061
Top 10 matches (any order): 0.066


Very slightly better for top 1 but worse on everything else. I think this is the best it can do from here. What if we initialize with the hand-rolled weights and see if it can improve that? 

In [ ]:
m3 = ModelSim()
m3.coeffs = torch.nn.Parameter(torch.tensor([0.001, 0.01, 0.01, 0.9, 3, 0.05]).unsqueeze(dim=1).unsqueeze(dim=2))
m3.coeffs.shape

torch.Size([6, 1, 1])

In [ ]:
output_dir = exp10.output_dir / 'learn_coefficients'
output_dir.mkdir(exist_ok=True)
checkpointer = CheckPointer(output_dir, 'coeff_model_hand_rolled_checkpoint', start_num=0)

In [ ]:
trainer = Trainer(
    model=m3,
    checkpointer=checkpointer,
    get_batch_func=get_batch_func,
    estimate_loss_func=estimate_loss_func,
    iters_trained=0
)

In [ ]:
# Get a starting point for the loss
estimate_loss_func(m3)

{'train': tensor(79.1921), 'val': tensor(77.6637)}

In [ ]:
# Begin with a moderate learning rate
learning_rate = 3e-4
optimizer = torch.optim.AdamW(m3.parameters(), lr=learning_rate)

trainer.train(1000, optimizer, eval_interval=500)

  0%|          | 0/1000 [00:00<?, ?it/s]

step 499: train loss 77.9316, val loss 76.6000
step 999: train loss 78.6018, val loss 77.1291


It got slightly better then slightly worse. Let's go back to the first checkpoint of this training run. 

In [ ]:
checkpoint = torch.load(checkpointer.output_dir / 'coeff_model_hand_rolled_checkpoint_000000.pt')
checkpoint['iters'], checkpoint['train_loss'], checkpoint['val_loss']

(500, tensor(77.9316), tensor(76.6000))

In [ ]:
m3.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
m3.coeffs.data

tensor([[[ 0.0184]],

        [[-0.0071]],

        [[-0.0237]],

        [[ 0.8463]],

        [[ 3.0463]],

        [[ 0.0057]]])

In [ ]:
total_probs, _ = m3(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.815
Top 1 matches (any order): 0.815
Top 2 matches: 0.499
Top 2 matches (any order): 0.559
Top 3 matches: 0.340
Top 3 matches (any order): 0.414
Top 4 matches: 0.274
Top 4 matches (any order): 0.341
Top 5 matches: 0.210
Top 5 matches (any order): 0.258
Top 6 matches: 0.178
Top 6 matches (any order): 0.207
Top 7 matches: 0.131
Top 7 matches (any order): 0.150
Top 8 matches: 0.109
Top 8 matches (any order): 0.125
Top 9 matches: 0.095
Top 9 matches (any order): 0.106
Top 10 matches: 0.062
Top 10 matches (any order): 0.066


Very similar to what the model got to from random initialization. Let's try one more time with a smaller learning rate.

In [ ]:
learning_rate = 3e-5
optimizer = torch.optim.AdamW(m3.parameters(), lr=learning_rate)

# Give it a lot more time to run
trainer.train(10000, optimizer, eval_interval=500)

  0%|          | 0/10000 [00:00<?, ?it/s]

step 499: train loss 77.8812, val loss 76.5227
step 999: train loss 77.9839, val loss 76.7161
step 1499: train loss 78.1107, val loss 77.1959
step 1999: train loss 77.7038, val loss 76.5574
step 2499: train loss 77.8688, val loss 76.9922
step 2999: train loss 78.2711, val loss 76.3786
step 3499: train loss 78.3931, val loss 76.5468
step 3999: train loss 77.9371, val loss 76.7095
step 4499: train loss 77.7700, val loss 76.4692
step 4999: train loss 78.1189, val loss 77.1611
step 5499: train loss 77.3637, val loss 76.8556
step 5999: train loss 78.4040, val loss 76.5145
step 6499: train loss 77.7374, val loss 76.2394
step 6999: train loss 77.5930, val loss 76.9928
step 7499: train loss 78.1478, val loss 76.7629
step 7999: train loss 77.6600, val loss 76.6683
step 8499: train loss 77.6980, val loss 76.7173
step 8999: train loss 78.3584, val loss 76.5446
step 9499: train loss 78.1036, val loss 76.6569
step 9999: train loss 78.4474, val loss 76.7377


In [ ]:
m3.coeffs.data

tensor([[[ 0.0178]],

        [[-0.0051]],

        [[-0.0226]],

        [[ 0.7633]],

        [[ 3.1019]],

        [[ 0.0050]]])

In [ ]:
total_probs, _ = m3(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.814
Top 1 matches (any order): 0.814
Top 2 matches: 0.498
Top 2 matches (any order): 0.559
Top 3 matches: 0.340
Top 3 matches (any order): 0.414
Top 4 matches: 0.274
Top 4 matches (any order): 0.341
Top 5 matches: 0.210
Top 5 matches (any order): 0.258
Top 6 matches: 0.178
Top 6 matches (any order): 0.208
Top 7 matches: 0.130
Top 7 matches (any order): 0.151
Top 8 matches: 0.110
Top 8 matches (any order): 0.125
Top 9 matches: 0.095
Top 9 matches (any order): 0.106
Top 10 matches: 0.063
Top 10 matches (any order): 0.066


Very slightly worse.

## Cosine Sims for Length 256 Strings


In [ ]:
strings256 = all_unique_substrings(ts.text, 256)

In [ ]:
torch.manual_seed(1337)
sample_size = 500
indices256 = torch.randperm(len(strings256))[:sample_size]
strings256_sample = [strings256[i.item()] for i in indices256]

In [ ]:
model_outputs_sample256 = get_model_outputs(strings256_sample, encoding_helpers)

In [ ]:
prompts_exp256 = BlockInternalsExperiment(encoding_helpers, accessors, strings256_sample)

In [ ]:
class CosineSimilaritiesForFinalFFWDExperiment:
    def __init__(
        self,
        exp: FinalFFWDExperiment,
        output_folder: Path,
    ):
        self.exp = exp
        self.output_folder = output_folder

        self.n_batches = exp.n_batches

    def ffwd_out_sims_filename(self, batch_idx: int, block_idx: int):
        return self.output_folder / f'ffwds_out_sims_{batch_idx:04d}_{block_idx:02d}.pt'

    def generate_ffwd_out_sims(self, get_queries: Callable[[int], torch.Tensor], disable_progress_bar=False):
        block_idx = n_layer - 1
        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            queries = get_queries(block_idx)
            assert queries.dim() == 2
            n_queries = queries.shape[0]

            ffwd_out_batch = torch.load(str(self.exp._ffwd_output_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True)
            batch_size = ffwd_out_batch.shape[0]
            sims = F.cosine_similarity(
                ffwd_out_batch.reshape(batch_size, 1, -1).expand(-1, n_queries, -1),
                queries,
                dim=-1
            )
            torch.save(sims, str(self.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)))




In [ ]:
ffwd_exp256 = FinalFFWDExperiment(
    eh=encoding_helpers,
    accessors=accessors,
    strings=strings256,
    output_dir=Path('../artifacts/block_internals_results/large_files/slen256'),
    batch_size=400,
)

In [ ]:
output_folder = ffwd_exp256.output_dir / 'cosine_sims'
output_folder.mkdir(exist_ok=True)

In [ ]:
cos_exp256 = CosineSimilaritiesForFinalFFWDExperiment(ffwd_exp256, output_folder)

In [ ]:
cos_exp256.generate_ffwd_out_sims(get_queries=lambda block_idx: prompts_exp256.ffwd_output(block_idx=block_idx)[:, -1, :])

  0%|          | 0/2788 [00:00<?, ?it/s]

In [ ]:
next_token_map256 = build_next_token_map(ts.text, 256, tokenizer.vocab_size, tokenizer.stoi)

In [ ]:
block_idx = n_layer - 1
ffwd256 = filter_across_batches(
    get_batch=lambda batch_idx: torch.load(str(cos_exp256.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp256.n_batches,
    filter_fn=lambda batch: batch > 0.91,
    n_queries=sample_size,
)
filter_result_stats(ffwd256)

{'min': 1, 'max': 38780, 'mean': 1561.9, 'std': 4162.8430373964375}

In [ ]:
ffwd256_strings = get_matching_strings(ffwd256, strings256)

In [ ]:
ffwd256_freqs = [
    torch.stack([
        next_token_map256[matching_string]
        for matching_string in matching_strings
    ]).sum(dim=0)
    for matching_strings in ffwd256_strings
]
ffwd256_probs = [
    freqs / freqs.sum()
    for freqs in ffwd256_freqs
]

In [ ]:
topn_matches, topn_matches_any_order = analyze_simulate_results(ffwd256_probs, model_outputs_sample256)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

Top 1 matches: 0.802
Top 1 matches (any order): 0.802
Top 2 matches: 0.362
Top 2 matches (any order): 0.412
Top 3 matches: 0.246
Top 3 matches (any order): 0.298
Top 4 matches: 0.200
Top 4 matches (any order): 0.250
Top 5 matches: 0.156
Top 5 matches (any order): 0.154
Top 6 matches: 0.128
Top 6 matches (any order): 0.148
Top 7 matches: 0.070
Top 7 matches (any order): 0.086
Top 8 matches: 0.044
Top 8 matches (any order): 0.068
Top 9 matches: 0.032
Top 9 matches (any order): 0.038
Top 10 matches: 0.030
Top 10 matches (any order): 0.016


In [ ]:
idx = 0
print(f"Query {idx} ({repr(strings256_sample[idx])}): ")
result = ffwd256[idx]
for j in result[:20]:
    print(f"  {repr(strings256[j])}")


Query 0 (" see my shame in him.\nThou art a widow; yet thou art a mother,\nAnd hast the comfort of thy children left thee:\nBut death hath snatch'd my husband from mine arms,\nAnd pluck'd two crutches from my feeble limbs,\nEdward and Clarence. O, what cause have I,\nThin"): 
  " so.\n\nSICINIUS:\nLet them assemble,\nAnd on a safer judgment all revoke\nYour ignorant election; enforce his pride,\nAnd his old hate unto you; besides, forget not\nWith what contempt he wore the humble weed,\nHow in his suit he scorn'd you; but your loves,\nThin"
  "I'ld have beaten him like a dog, but for\ndisturbing the lords within.\n\nAUFIDIUS:\nWhence comest thou? what wouldst thou? thy name?\nWhy speak'st not? speak, man: what's thy name?\n\nCORIOLANUS:\nIf, Tullus,\nNot yet thou knowest me, and, seeing me, dost not\nThin"
  "e not--to save my life, for if\nI had fear'd death, of all the men i' the world\nI would have 'voided thee, but in mere spite,\nTo be full quit of those my banishers,\nStand I bef

In [ ]:
class TensorBatchIterator:
    def __init__(self, n_batches: int, get_batch: Callable[[int], torch.Tensor]):
        self.n_batches = n_batches
        self.get_batch = get_batch

        self.next_batch_idx = 0
        self.current_batch: Optional[torch.Tensor] = None
        self.idx_within_batch = 0

        self._load_next_batch()

    def _load_next_batch(self):
        if self.next_batch_idx >= self.n_batches:
            raise StopIteration()

        self.current_batch = self.get_batch(self.next_batch_idx)
        self.idx_within_batch = 0
        self.next_batch_idx += 1

    def __next__(self):
        if self.current_batch is None:
            raise StopIteration()

        if self.idx_within_batch >= self.current_batch.shape[0]:
            self._load_next_batch()
            if self.current_batch is None:
                raise StopIteration()

        result = self.current_batch[self.idx_within_batch, :]
        self.idx_within_batch += 1
        return result

class EmbeddingCosineSims:
    def __init__(self, exp: CosineSimilaritiesExperiment):
        self.exp = exp

    def __iter__(self):
        return TensorBatchIterator(
            n_batches=self.exp.n_batches,
            get_batch=lambda batch_idx: torch.load(str(self.exp.embedding_sims_filename(batch_idx=batch_idx)), mmap=True)
        )



In [ ]:
emb_sims = EmbeddingCosineSims(cos_exp)
for i, sim in enumerate(emb_sims):
    print(i, sim.shape)

In [ ]:
sims = next(iter(emb_sims))
sims.shape

torch.Size([500])